# <font color=red> Data Exploration
#### This notebook aims to find out useful features 
### <font color=blue> Rules :
#### Since we are a lot contributing to the notebook, please comment your code and use explicit variable names
#### Do not modify the raw data as others would like to work on it
#### Only modify your section (copy-paste others code if needed) so that no clashes occur when pulling/pushing on remote repository

## <font color=green>Hugo

In [1]:
# Specific imports in your section

## <font color=green>Nicolas

In [2]:
# Specific imports in your section

## <font color=green>Mohamed

In [1]:
import pandas as pd
import numpy as np
import sklearn as skl
import nltk
from tokenizer import Tokenizer
import seaborn as sns
import time

#### Getting the data

In [2]:
train_fname = 'train.csv'
#test_fname = 'test.csv'
X = pd.read_csv(train_fname,header=None)
y = pd.read_csv(train_fname, header=None)[0]
#X_test = pd.read_csv(test_fname, header=None)[0]

In [3]:
X.columns = ['label', 'content']

In [4]:
y.value_counts()

0    3066
1    1349
Name: 0, dtype: int64

In [5]:
index_1 = [index for index in range(X.shape[0]) if y[index]==1]
index_1[0:10]

[6, 8, 11, 12, 15, 18, 23, 27, 28, 29]

#### Some insulting comments

In [6]:
X.content[index_1[:10]] # Wow violent lui ... hahaa

6     "who told you that....stupid ask yourself who ...
8                             "You're an idiot baldo. "
11    "You show your stupidity and ignorance each an...
12    "Wait until you are a minority in your own red...
15    "Its "faggot" you ignorant fuck plus what you ...
18     "Get thatdick outta your mouth so you can talk."
23    "Judging by your prior posts listed in your pr...
27    "RYAN...........GO BACK TO SUCKING YOUR MOTHER...
28                                  "Your are an idiot"
29    "Very sorry I mentioned it.\xa0 Just keep bitc...
Name: content, dtype: object

#### Adding some features

In [7]:
X['char_count'] = np.array([len(x) for x in X.content])

In [8]:
X.shape

(4415, 3)

In [9]:
X.head()

,label,content,char_count
0,0,"""Imagine being able say, you know what, no san...",684
1,0,"""""But Jack from Raleigh wasn't done. He came b...",365
2,0,"""the Star box allows you to link your comment ...",175
3,0,"""Cheney,Rush,Nugent.The list is endless.""",41
4,0,"""Obama....I'm blow'n mo smoke up yo arses........",181


## Tokenizing Stemming and Cleaning a sentence

In [10]:
# To Keep updated
TOKENS = [
    ### Replacements first ###
    ('WHITE_SPACE'     , r"(\\\\xc2|\\\\xa0|\\xa0|\\xa1|\\xa3|\\xa9|\\r|\\ufeff|\\u2013|\\u2016|\\u200f|\\u2665|\\U0001f308|\\U0001f3e9|\\U0001f48b|\\\\|\\|&nbsp|&amp|=|\"\"|\"|\\u2018|\\u2019)", r" "),
    ('LETTER_A'        , r"(\\xe1|\\xe3|\\xe0|\\xc2|\\u1ef1|\\u0105|\\u1eb7|\\xe5|\\xe4)", r"a"),
    ('LETTER_E'        , r"(\\xe9|\\xe8|\\u1ec3|\\u1ec5|\\u1ebf|\\u0119)", r"e"),
    ('LETTER_I'        , r"(\\xed)", r"i"),
    ('LETTER_O'        , r"(\\x00 |\\xf8|\\xf6|\\xf3)", r"o"),
    ('LETTER_U'        , r"(\\xfa|\\xdc|\\xfc|\\u1ee9|\\u0169)", r"u"),
    ('LETTER_Y'        , r"(\\xfd)", r"u"),
    ('LETTER_C'        , r"(\\xe7|\\u0107)", r"c"),
    ('LETTER_Z'        , r"(\\u017a|\\u017e|\\u017c)", r"z"),
    ('PONCT_!'        , r"(\\u203d)", r"!"),
    ### Tokens ###
    ('PUNCT'      , r"(!+|\?+|\.+)", r" \1 "),
    ('NIL'          , r"\'|,", r" "),
    ('SMILEY'       , r"(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$)|xD|XD", r" \0 "),
    ('PUNCT'      , r"(;+|\,+|\)+|\(+)", r" \1 "),
    ('BREAK'        , r"(\\n|\\\n)", r" "),
#    ('QUOTE'      , r"(\"|\')+(\w+)(\"|\')+", "QUOTE"), # A améliorer pour mettre la citation entre deux <QUOTE>
    ('BEGIN_WHITESPACE' , r"^\s+", r""),
    ('END_WHITESPACE' , r"\s+$", r""),
    ('WHITESPACES' , r"\s+", r" ")
]

# Ca déconne
INSULTS = [
    ### Insults Replacment ###
    ('FUCK'    , r"f(.*ck|u.*k|uc.*)(.?ng*)", r" fuck "),# fuck/fucking style
    ('FUCK_2'    , r"f(.*ck|u.*k|uc.*)((\s)*you|.*er)*", r" fuck you "),#fuck you + fuck your XXXer
    ('FUCK_3'    , r"(\w+-)*f(.*ck|u.*k|uc.*)((\s)*you|.?ng|.*er)*", r" mother fuck "),# compouned fuck # mother fuck plutot ?
    # Replace above regex "mother fuck" by "you fuck" since personal insult ?
    ('ASSHOLE',  r"a.*ole*s*", r" asshole "),# ass hole and its versions
    ('STALK',  r"stalk(in*g*|ers?)?", r" stalk "),
    ('STUPID',  r"(\$|s)tupi?d", r" stupid "),
    ('SHIT',  r"sh?(.*ty*|i.*)", r" shit ")
]

RANDOM = [
    ### Random Replacement ###
    ('LAUGH',  r"\b(?:a*(?:ha)+h?|(?:l+o+)+l+)\b", r" laugh "), # matches laughs such as "hahaha" or "lolll"
    ('DOLLAR',  r"[0-9]+k?\s?\$?", r" dollar "),
    ('DATE',  r"[0-9]+th(\s?of)?(\s\w+)?", r" date "),# eg : 18th or "18th of Month"
    ('YEAR',  r"[0-9]{4}?", r" year "),
    ('NUMBER',  r"[0-9]{2}", r" number "),
    ('LOVE',  r"<3", r" love "),
    ('PSEUDO',  r"@\s?\w+", r" pseudo "),
    ('URL',  r"(http:)?(\/\/)?((www\.)?\w+\.\w+)\/?(\w+.\w+)?", r" url ")   
]


# <font color=red>UTILISER LES DEUX DICO AU DESSUS

In [11]:
myTokenizer = Tokenizer(TOKENS)
#myReplacer = Tokenizer(REPLACERS)

In [12]:
X.content.values.shape

(4415,)

In [13]:
type(X.content.values), type(X.content.values[0])

(numpy.ndarray, str)

In [14]:
X.content.values[1]

'""But Jack from Raleigh wasn\'t done. He came back with this bit of furious grammatical genius:"\\n"Holy hell, Jack. Calm down."\\n\\nGOD D@MN HILARIOUS!\\n\\nWho writes your material GraziD?  \\n\\nMM never even acknowledged we were here (well accept when Uber ticked him off)  GraziD not only interacts with us, he calls you dumb when you\'re being dumb... right beeaner?"'

In [15]:
tokens = myTokenizer.tokenize(X.content.values[1])

In [16]:
tokens[:12], len(tokens)

(['But',
  'Jack',
  'from',
  'Raleigh',
  'wasn',
  't',
  'done',
  '.',
  'He',
  'came',
  'back',
  'with'],
 75)

In [17]:
new_string = myTokenizer.reconstruct(tokens)

In [18]:
new_string

'but jack from raleigh wasn t done . he came back with this bit of furious grammat geniu \x00 n holi hell jack . calm down . n ngod d@mn hilari ! n nwho write your materi grazid ? n nmm never even acknowledg we were here ( well accept when uber tick him off ) grazid not only interact with us he call you dumb when you re being dumb ... right beeaner ?'

## Tokenizing Stemming and Cleaning the DataSet

In [19]:
X_list = X.content.values
X_list[:5]

array([ '"Imagine being able say, you know what, no sanctions, no forever hearings on IEAA regulations, no more hiding\\xa0under\\xa0the pretense of friendly nuclear energy. \\xa0You have 2 days to; \\xa0i.e. \\xa0let in the inspectors, quit killing the civilians, respect the border and rights of your neighboring country, \\xa0or we ( whoever we are) will shut off your nuclear plant, your monitoring system and whatever else we fancy, like your water\\xa0treatment\\xa0plants and early warning sandstorm system and the traffic lights of all major cities...\\xa0\\nand yes..( pinky finger to lip edge) so your teenagers revolt and topple your regime... \\xa0disconnect ... FACEBOOK.... buwhahjahahaha."',
       '""But Jack from Raleigh wasn\'t done. He came back with this bit of furious grammatical genius:"\\n"Holy hell, Jack. Calm down."\\n\\nGOD D@MN HILARIOUS!\\n\\nWho writes your material GraziD?  \\n\\nMM never even acknowledged we were here (well accept when Uber ticked him off)  GraziD

In [20]:
X_list.shape

(4415,)

In [21]:
tokens_data = myTokenizer.tokenize_array(X_list)

In [22]:
len(tokens_data)

4415

In [23]:
new_X_list = []
for tokens in tokens_data:
    new_X_list.append(myTokenizer.reconstruct(tokens))

In [24]:
new_X_list[:5]

['imagin being abl say you know what no sanction no forev hear on ieaa regul no more hide under the pretens of friend nuclear energi . you have 2 day t \x00 i . e . let in the inspector quit kill the civilian respect the border and right of your neighbor countri or we ( whoever we are ) will shut off your nuclear plant your monitor system and whatev els we fanci like your water treatment plant and earli warn sandstorm system and the traffic light of all major citi ... nand yes .. ( pinki finger to lip edg ) so your teenag revolt and toppl your regim ... disconnect ... facebook .... buwhahjahahaha .',
 'but jack from raleigh wasn t done . he came back with this bit of furious grammat geniu \x00 n holi hell jack . calm down . n ngod d@mn hilari ! n nwho write your materi grazid ? n nmm never even acknowledg we were here ( well accept when uber tick him off ) grazid not only interact with us he call you dumb when you re being dumb ... right beeaner ?',
 'the star box allow you to link you

In [25]:
len(new_X_list)

4415

In [26]:
X_new = np.asarray(new_X_list)

In [27]:
X_new.shape

(4415,)

In [28]:
X['processed'] = X_new

In [29]:
X.head()

,label,content,char_count,processed
0,0,"""Imagine being able say, you know what, no san...",684,imagin being abl say you know what no sanction...
1,0,"""""But Jack from Raleigh wasn't done. He came b...",365,but jack from raleigh wasn t done . he came ba...
2,0,"""the Star box allows you to link your comment ...",175,the star box allow you to link your comment to...
3,0,"""Cheney,Rush,Nugent.The list is endless.""",41,cheney rush nugent . the list is endless .
4,0,"""Obama....I'm blow'n mo smoke up yo arses........",181,obama .... i m blow n mo smoke up yo ars ........


# <font color=red> TF-IDF

In [43]:
from sklearn import cross_validation

In [44]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X.processed, X.label, test_size=0.4, random_state=0)

In [45]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [46]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

In [47]:
X_train_counts.shape

(2649, 9294)

In [48]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

In [49]:
X_train_tf.shape

(2649, 9294)

## Classifiers

In [64]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', LogisticRegression())
                    ])
                    
_ = text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)

In [65]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

             precision    recall  f1-score   support

          0       0.79      0.96      0.86      1211
          1       0.82      0.43      0.57       555

avg / total       0.80      0.79      0.77      1766



### NAIVE-BAYES : ACCURACY = 0.78

### LOGISTIC REGRESSION = 0.80

## Adding general Features

In [6]:
class BadWordCounter():
    def __init__(self):
        with open("my_badlist.txt") as f:
            badwords = [l.strip() for l in f.readlines()]
        self.badwords_ = badwords

    def get_feature_names(self):
        return np.array(['n_words', 'n_chars', 'allcaps', 'max_len',
            'mean_len', '@', '!', 'spaces', 'bad_ratio', 'n_bad',
            'capsratio'])

    def fit(self, documents, y=None):
        return self

    def transform(self, documents):
        ## some handcrafted features!
        n_words = [len(c.split()) for c in documents]
        n_chars = [len(c) for c in documents]
        # number of uppercase words
        allcaps = [np.sum([w.isupper() for w in comment.split()])
               for comment in documents]
        # longest word
        max_word_len = [np.max([len(w) for w in c.split()]) for c in documents]
        # average word length
        mean_word_len = [np.mean([len(w) for w in c.split()])
                                            for c in documents]
        # number of google badwords:
        n_bad = [np.sum([c.lower().count(w) for w in self.badwords_])
                                                for c in documents]
        exclamation = [c.count("!") for c in documents]
        addressing = [c.count("@") for c in documents]
        spaces = [c.count(" ") for c in documents]

        allcaps_ratio = np.array(allcaps) / np.array(n_words, dtype=np.float)
        bad_ratio = np.array(n_bad) / np.array(n_words, dtype=np.float)

        return np.array([n_words, n_chars, allcaps, max_word_len,
            mean_word_len, exclamation, addressing, spaces, bad_ratio, n_bad,
            allcaps_ratio]).T

In [5]:
bad_counter = BadWordCounter()

In [6]:
features = bad_counter.transform(X.content)

In [7]:
features.shape

(4415, 11)

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score

In [22]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(features, y, test_size=0.4, random_state=0)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
clf_1 = SVC(kernel='linear', C=1)
clf_2 = LinearRegression()
cross_validation.cross_val_score(clf_1, features, y, cv=5)